In [1]:
%pylab widget
%load_ext autoreload
%autoreload 2

from analysis_functions import *
import fastcat.fastcat as fc

from scipy.optimize import minimize

Populating the interactive namespace from numpy and matplotlib


In [2]:
image_fc_mv = np.load('fc_images/fc_image_gos_6x_493_noise.npy')
image_bcca_mv = np.load('data/image_bcca_mv_6x.npy')

In [3]:
image_fc_kv = np.load('fc_images/fc_image_kv_887_noise.npy')
image_bcca_kv = np.load('/home/xcite/MV_CBCT/recon_kv_jan_2021.npy')
image_bcca_kv = np.rot90(image_bcca_kv.T[50],3) 

In [4]:
recon_slice = image_fc_mv
im = create_mask(recon_slice.shape,r=5.7,radius=0.3)
contrast = []
CNR_fc = []
noise = []

for ii in range(2,int(np.max(im)+1)):
    
    contrast.append(np.mean(recon_slice[im == ii]))
    CNR_fc.append(contrast[-1]/np.sqrt(np.std(recon_slice[im == ii])**2 + np.std(recon_slice[im == 6])**2))
    noise.append(np.std(recon_slice[im == ii]))#/np.sqrt(np.sum(recon_slice[im == ii])))

cc_kv = np.array(sorted(contrast))
CNR_fc_kv = np.array(CNR_fc)[np.argsort(contrast)]
fc_noise_kv = np.array(noise)[np.argsort(contrast)]

recon_slice = image_bcca_mv
im = create_mask(recon_slice.shape,off =[-0.,0], r= 5.75,radius=0.3)
contrast = []
CNR = []
noise = []

for ii in range(2,int(np.max(im)+1)):
    
    contrast.append(np.mean(recon_slice[im == ii]))
    CNR.append(contrast[-1]/np.sqrt(np.std(recon_slice[im == ii])**2 + np.std(recon_slice[im == 6])**2))
    noise.append(np.std(recon_slice[im == ii]))#/np.sqrt(np.sum(recon_slice[im == ii])))
    
bcca_data_kv = np.array(sorted(contrast))
CNR_bcca_kv = np.array(CNR)[np.argsort(contrast)]
bcca_noise_kv = np.array(noise)[np.argsort(contrast)]

image_fc_kv2 = np.load('data/fc_kv_image_no_noise.npy')
recon_slice = image_fc_kv2
recon_slice2 = image_fc_kv
im = create_mask(recon_slice.shape,r=5.75,radius=0.3)
contrast = []
CNR = []
noise = []

for ii in range(2,int(np.max(im)+1)):
    contrast.append(np.mean(recon_slice[im == ii]))
    CNR.append(contrast[-1]/np.sqrt(np.std(recon_slice[im == ii])**2 + np.std(recon_slice[im == 3])**2))
    noise.append(np.std(recon_slice[im == ii]))#/np.sqrt(np.sum(recon_slice[im == ii])))

# Get rid of the delrin
cc_mv = np.array(sorted(contrast))[np.arange(len(contrast))]
cc_CNR_mv = np.array(CNR)[np.argsort(contrast)]
fc_noise_mv = np.array(noise)[np.argsort(contrast)]


image_bcca_kv2 = np.load('/home/xcite/MV_CBCT/recon_kv_jan_2021.npy')
recon_slice = np.rot90(image_bcca_kv2.T[50],3).copy()
im = create_mask(recon_slice.shape,r=5.8,off = [0.2,0.05],rot = 1,radius=0.3)
contrast = []
CNR = []
noise = []

for ii in range(2,int(np.max(im)+1)):
    contrast.append(np.mean(recon_slice[im == ii]))
    CNR.append(contrast[-1]/np.sqrt(np.std(recon_slice[im == ii])**2 + np.std(recon_slice[im == 3])**2))
    noise.append(np.std(recon_slice[im == ii]))#/np.sqrt(np.sum(recon_slice[im == ii])))
    
# Get rid of the delrin
bcca_noise_mv = np.array(noise)[np.argsort(contrast)]
bcca_data_mv = np.array(sorted(contrast))
CNR_data_mv = np.array(CNR)[np.argsort(contrast)]

In [5]:
import matplotlib.font_manager
# matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

In [71]:
plt.close('all')
plt.figure(27,figsize=[7,7])

x = [.853,0.945,.998,1.147,1.36,1.87]

ax = plt.subplot(221)
bcca_data = np.load('data/contrast_bcca_kv.npy')[2:]
water = 0.020#(bcca_data[4]+0.0003)
air = 0.0056
yerr_bcca = (bcca_noise_mv)/(air-water)*-1000
yerr_fc = (fc_noise_mv)/(air-water)*-1000
plt.errorbar(x,(bcca_data-water)/(air-water)*-1000,yerr = yerr_bcca,fmt='*',color='darkorange',label='')
plt.errorbar(np.array(x) + 0.02,(cc_mv-water)/(air-water)*-1000,fmt='.',yerr = yerr_fc,color='cornflowerblue',label='fastCAT')
circle1 = plt.text(1.35,1180,'HU Comparison',color='k',size='x-large',ha='center');ax.add_artist(circle1)

add = 0
water = 0.0195
air = 0.0054
# print((bcca_data-(water-add))/(air-(water-add))*-1000,(cc_mv-(water-add))/(air-(water-add))*-1000)
print(1.2*yerr_bcca,yerr_fc)
# circle1 = plt.text(2.55,1180,'Detector MTF',color='k',size='x-large');ax.add_artist(circle1)
# circle1 = plt.text(0.85,1000,'c)',color='k',size='x-large');ax.add_artist(circle1)
# plt.xticks([])
plt.legend(['Varian OBI 100 kVp','Fastcat 100 kVp'],loc=4)
plt.ylabel('HU')
plt.grid('on')
ax = plt.subplot(223)
cc_kv = np.load('data/mv_data_revised_delrin.npy')
bcca_data_kv = np.load('data/mv_data_revised_delrin_bcca.npy')
water = 0.00425 #(bcca_data_kv[2]+bcca_data_kv[3])/2
air = 0.001861
yerr_bcca = (bcca_noise_kv)/(air-water)*-725
yerr_fc = (fc_noise_kv+0.0001)/(air-water)*-725
plt.errorbar(np.array(x),(bcca_data_kv - water)/(air - water)*-725,fmt='*',yerr = 1.2*yerr_bcca,color='darkorange',label='')
plt.errorbar(np.array(x)+0.02,(cc_kv - water)/(air-water)*-725,fmt='.',yerr = yerr_fc,color='cornflowerblue',label='Fastcat')
# circle1 = plt.text(0.85,900,'f)',color='k',size='x-large');ax.add_artist(circle1)
plt.legend(['Varian 6 MV','Fastcat 6 MV'],loc=4)
plt.grid('on')
plt.xlabel('Relative Electron Density')
plt.ylabel('HU')
# plt.tight_layout()
print(1.2*yerr_bcca,yerr_fc)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ 96.08584  108.209854  97.52981   96.47045  110.91382   85.321976] [103.21559   91.058395  93.78461   97.96999  102.16253  102.092735]
[124.02515 136.4409  131.74757 129.53842 121.21525 115.90237] [129.61066  126.38186  135.44427  124.865875 127.77726  143.84915 ]


In [61]:
plt.subplot(222)
# plt.ylim([0.2,1.05])

xx = -0.07
yy = 1.07

speks = ['--','-','-.','--',':']
colors = ['cornflowerblue','darkorange','darkgrey','crimson']

# plt.text(xx,yy,'(b)',color='cornflowerblue',fontsize=14)

mtfs = np.load('../paper_2/csi_file.npy')

ii = jj = 0
#     plt.plot(mtfs[ii][0][::5],mtfs[ii][1][::5],'.',color=colors[jj],markersize=1.5,alpha=0.8)
# plt.title('CsI Validation 60 kVp 0.150 mm pixel')
plt.ylabel('MTF')
# plt.yticks(ticks=[0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],labels=[])
# plt.xlabel('Spatial frequency [1/mm]')
plt.grid('on')

dats2 = np.array([0.026052635192249713, 0.994492130588933,
0.15568447352428327, 0.9615179753726617,
0.2678324444565603, 0.9144254069133888,
0.39290299807856766, 0.8602696668771996,
0.4920586031058045, 0.8137678767748149,
0.6258139950696089, 0.7613757489073764,
0.7293278790894058, 0.7172267633484641,
0.8673971425451179, 0.6630686443390259,
1.0228360826844245, 0.6124377496670886,
1.2170998662509018, 0.5459129347171532,
1.385524828366584, 0.49410286230468914,
1.55832710126082, 0.4464107925866142,
1.6533781844420798, 0.421092173286313,
1.9127053003923926, 0.3610278566901599,
2.2542687636215324, 0.2927108818924834,
2.5224075964869335, 0.24617816513785862,
2.7689834158214595, 0.20906380347687215,
3.037261815117482, 0.1754758731623035,
3.2926176317237736, 0.1489511144246512,
3.608602374577365, 0.11947325689364152,
3.8727064688120985, 0.10059610416112896,
4.197452177151463, 0.07994265140253631,
4.522242292991479, 0.06340799432941657,
4.8557426228879494, 0.05099054695960281,
5.27588198655646, 0.03679204161721117,
])

ax = plt.gca()
circle5 = plt.text(1.6,1.15,'MTF Comparison',color='k',size='x-large',ha='center');ax.add_artist(circle5)
# plt.plot(np.array(dats[::2]),dats[1::2])

# plt.plot(np.array(dats3[::2]),dats3[1::2]/100)
plt.plot(np.array(dats2[::2]),dats2[1::2],'o-',c='darkorange',linewidth= 1.1,markersize=3,label='Howansky et al. Measured')
plt.plot(np.array(dats2[::2]),dats2[1::2],'.',c='darkorange',linewidth= 1.1,markersize=4)
plt.plot(mtfs[0],mtfs[1],speks[jj],color=colors[jj],linewidth= 1.5,markersize=1.5,alpha=0.8,label='Fastcat 100 kVp')
plt.legend()
plt.xlim([0,3.33])
plt.ylim([0,1.1])
plt.tight_layout()
# plt.savefig('CsI_validation',dpi=300)

In [62]:
# plt.figure()
# plt.plot(CNR_data_mv)
# plt.plot(cc_CNR_mv,'r')

In [63]:
# plt.close('all')

In [64]:
# import matplotlib.pyplot as plt
# import pickle

# with open('../paper_2/shi.pkl', 'rb') as fid:
#     ax = pickle.load(fid)
# plt.show()

In [65]:
import sys
import os
import tigre
from scipy.signal import fftconvolve
from scipy.ndimage import convolve


sys.path.insert(0,'../xpecgen')

import fastcat.fastcat as fc

dose = 1

phan_map_hard =  ['air','water','Spongiosa_Bone_ICRP','G4_BONE_COMPACT_ICRU',
             'G4_BONE_CORTICAL_ICRP','C4_Vertebra_ICRP','D6_Vertebra_ICRP','G4_B-100_BONE']

phan_map_soft =  ['air','water','G4_ADIPOSE_TISSUE_ICRP','G4_BRAIN_ICRP',
             'G4_LUNG_ICRP','G4_MUSCLE_SKELETAL_ICRP','G4_TISSUE_SOFT_ICRP','pmma']

spectrum_files = ['Al_spectrum_25', 'C_spectrum_6', 'Al_spectrum_6', 'C_spectrum_25', 'W_spectrum_6', 'W_spectrum_25']

dets = ['CWO-392-micrometer','CuGOS-336-micrometer']

phan_maps = [phan_map_hard]#,phan_map_soft]

# angles = np.linspace(0,2*np.pi,90)

res = []

original_energies_keV = np.array([30, 40, 50 ,60, 70, 80 ,90 ,100 ,300 ,500 ,700, 900, 1000 ,2000 ,4000 ,6000])


angles = np.linspace(0,2*pi,360)

geo = 'catphan_low_contrast_512'

phantom = fc.Catphan_MTF()

for phan_map in phan_maps[:1]:
    for spectrum_file in spectrum_files[-2:-1]:
        for det in dets[1:]:
            s = fc.Spectrum()

            energies = []
            fluence = []

            with open(os.path.join(fc.data_path, "MV_spectra", f'{spectrum_file}.txt')) as f:
                for line in f:
                    energies.append(float(line.split()[0]))
                    fluence.append(float(line.split()[1]))

            # Check if MV

            s.x = np.array(energies)*1000  # to keV
            s.y = np.array(fluence)

            spectrum = fc.Spectrum()

            spectrum.x = energies
            spectrum.y = fluence

            kernel = fc.Kernel(s, det)

(19, 81, 81) 19


In [66]:
# Lets make a fake setup
# Very high res and then I'll bin it
# Then I'll convolve it

In [67]:
plot_stuff = True
plot_stuff_res = True

In [68]:
from scipy.interpolate import make_interp_spline, BSpline

In [69]:
h,w = 1024*4,2*1024 #Wouldn't change tbh for building lsf
step = 16*2 #Wouldn't change tbh for building lsf
pitch = kernel.pitch #mm
angle = 2.3 #deg
lsf_width = 0.3 #mm Wouldn't change tbh
nbins = 818

# --- Make a high res line ---

high_res = np.zeros([h,w])
Y, X = np.mgrid[:h, :w]
dist_from_line = np.abs((X - high_res.shape[1]/2) + Y*tan(angle*pi/180))
# The MTF is from a 0.3 mm pixel times the angle times 16 since it will be averaged over 32 pix
num_pix = lsf_width*1/cos(angle*pi/180)/pitch*16
high_res[dist_from_line < num_pix] = 1

if plot_stuff:
    plt.figure()
    plt.subplot(231)
    plt.imshow(high_res,cmap='bwr',interpolation='none')
    plt.savefig('LSF1')

# --- Average to make low res line ---
# Ugly sorry
low_res = np.array([[np.mean(high_res[ii:ii+step,jj:jj+step]) 
                     for ii in range(0,h,step)] for jj in range(0,w,step)]).T

if plot_stuff:
    plt.subplot(232)
    plt.imshow(low_res,cmap='bwr')
    plt.savefig('LSF2')

# --- Convlolve with the kernel ---
lsf_image = fftconvolve(low_res,kernel.kernel/np.sum(kernel.kernel),mode = 'same')

if plot_stuff:
    plt.subplot(233)
    plt.imshow(lsf_image,cmap='bwr')
    plt.savefig('LSF3')

# --- Pad and presample ---
pad_len = int((512 - lsf_image.shape[1])/2)
lsf_image =  np.pad(lsf_image,((0,0),(pad_len,pad_len)),mode='constant')
Y, X = np.mgrid[:lsf_image.shape[0], :lsf_image.shape[1]]
center = int(lsf_image.shape[1]/2)
# pitch needs to convert to cm from mm
dist_from_line = (X + Y*tan(angle*pi/180) - center + 0.5)*pitch/10

if plot_stuff:
    plt.figure()
    plt.plot(dist_from_line[10:-10,:],lsf_image[10:-10,:],'b.')
    plt.savefig('LSF4')

# --- Crop the convolved edges ---
inds = np.argsort(dist_from_line[10:-10,:].flatten())
line = dist_from_line[10:-10,:].flatten()[inds]
lsf = lsf_image[10:-10,:].flatten()[inds]
n,bins = np.histogram(line,nbins,weights=lsf,density=True)

n2 = np.load('../data/full_n.npy')
bins2 = np.load('../data/full_bins.npy') -0.0085

bins -= 0.0085

if plot_stuff:
    plt.figure()
    plt.plot(bins[1:],n/(np.sum(n)),'x',linewidth= 1.1,markersize=5,color='cornflowerblue')
    plt.plot(bins2[1:],n2/np.sum(n2),'ko',linewidth= 1.1,markersize=3)

    xnew = np.linspace(bins[1:].min(), bins[1:].max(), 5000) 
    spl = make_interp_spline(bins[1:], n/(np.sum(n)), k=3)  # type: BSpline
    power_smooth = spl(xnew)
    
    plt.plot(xnew,power_smooth,'-',linewidth= 1.1,markersize=5,color='cornflowerblue')
    
    xnew2 = np.linspace(bins2[1:].min(), bins2[1:].max(), 5000) 
    spl2 = make_interp_spline(bins2[1:], n2/(np.sum(n2)), k=3)  # type: BSpline
    power_smooth2 = spl2(xnew2)
    
    plt.plot(xnew2,power_smooth2,'k:',linewidth= 1.1,markersize=3)

    
#     plt.title('PSF from LSF Cu-GOS')
    plt.legend(['fastcat','Simulated'])
    plt.xlabel('Distance [mm]')
    plt.ylabel('Normalized amplitude [a.u.]')
    plt.xlim([-0.2,0.2])
    plt.tight_layout()
#     plt.savefig('LSF_good')
#     plt.savefig('LSF5')


mtf = np.absolute(np.fft.fft(power_smooth))
mtf_final = np.fft.fftshift(mtf)
N = len(mtf)
T = np.mean(np.diff(xnew))
xf = np.linspace(0.0, 1.0/(2.0*T), int((N-1)/2))
mm = np.argmax(mtf_final)
    
if plot_stuff_res:
    plt.figure(27)#,dpi=300,figsize=[8,4])
    plt.subplot(224)
    plt.grid('on')
    
    # plt.legend([])
    # plt.xlim([0,15])

    shi_data = [-0.0036600255852107244, 0.9985732893897139,
    0.028883896178828306, 0.9869637814824851,
    0.05821655345648036, 0.9552964008777066,
    0.08921020960640147, 0.9064175751164883,
    0.1218386383306087, 0.8532306428065635,
    0.17567540002389503, 0.7655438897096748,
    0.25229698659836164, 0.6677232950720785,
    0.38264460160796343, 0.5366967103480229,
    0.5064443842210934, 0.42720366468803805,
    0.6399828654853176, 0.326270882689769,
    0.7734630660873568, 0.25401218648646273,
    0.8857495038858628, 0.20908478962137955,
    0.9866304160947875, 0.17567598283051677,
    1.1102728409200182, 0.14360296881693158,
    1.2192635072719695, 0.1201951236569948,
    1.3510244283395543, 0.09382195840509122,
    1.4632584135420945, 0.07470123875547463,
    1.4925327901575614, 0.07170794394565894]

    shi_data_sim = [0, 1.0028606704405152,
    0.03089430894308942, 0.9856244321755537,
    0.04878048780487804, 0.9497821883662961,
    0.08130081300813002, 0.8995923311668645,
    0.11056910569105688, 0.8379458149882357,
    0.14308943089430892, 0.7834579635194632,
    0.1788617886178861, 0.7318307824912058,
    0.22113821138211376, 0.6816269480746382,
    0.26991869918699185, 0.6443077783213361,
    0.31707317073170727, 0.6170195913993523,
    0.36097560975609744, 0.5854380692804062,
    0.3967479674796748, 0.5553008595988538,
    0.4439024390243902, 0.517984019381741,
    0.4813008130081301, 0.4921424744333402,
    0.5235772357723576, 0.46342861136347735,
    0.5869918699186991, 0.43038647005381225,
    0.6585365853658535, 0.39160202203741235,
    0.7121951219512195, 0.36143918745777703,
    0.7756097560975608, 0.32696438139166484,
    0.8373983739837396, 0.2982225638875299,
    0.8926829268292682, 0.27378805879749335,
    0.9528455284552844, 0.25077922985533563,
    1.0130081300813008, 0.22490507140028415,
    1.1089430894308943, 0.19181633936683207,
    1.1788617886178858, 0.16879353320753832,
    1.2569105691056908, 0.1471917441237448,
    1.3382113821138208, 0.12701796072401972,
    1.3934959349593494, 0.11547743844200609,
    1.4585365853658534, 0.10249027418640944,
    1.4943089430894305, 0.09670836536445593]

    shi_sim_x = shi_data_sim[::2]
    shi_sim_y = shi_data_sim[1::2]

    shi_x = shi_data[::2]
    shi_y = shi_data[1::2]


    mtf = np.absolute(np.fft.fft(power_smooth2))
    mtf_final2 = np.fft.fftshift(mtf)
    N = len(mtf)
    T = np.mean(np.diff(xnew2))
    xf2 = np.linspace(0.0, 1.0/(2.0*T), int((N-1)/2))
    mm2 = np.argmax(mtf_final2)
    
#     plt.plot(xf2/10,mtf_final2[mm2+1:]/mtf_final2[mm2+1],'-.',linewidth= 1.1,markersize=2,color='darkorange')

    #     plt.plot(star_x,np.array(star_y)/star_y[0])
    plt.plot(shi_x,np.array(shi_y)/shi_y[0],'o-',c='darkorange',linewidth= 1.1,markersize=3)
#     plt.plot(shi_sim_x,np.array(shi_sim_y)/shi_sim_y[0],'.--',linewidth= 1.1,markersize=3,color='coral')

    plt.plot(xf/10,mtf_final[mm+1:]/mtf_final[mm+1],'--',linewidth= 1.5,markersize=2,color='cornflowerblue')
    plt.legend(['Shi et al. Measured','Fastcat 6 MV'])
    plt.xlim((0,1.5))
    plt.ylim([0,1.1])
    plt.xlabel('Spatial Frequency [1/mm]')
    plt.ylabel('MTF')
    plt.tight_layout()
    plt.savefig('/home/xcite/fastcat_paper_3/figures/CNR_MTF.pdf')
    
import pickle

ax = plt.gca()
with open('shi.pkl','wb') as fid:
    pickle.dump(ax, fid)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …